<a href="https://colab.research.google.com/github/sindhusatish707/Exploring_Data_Engineering/blob/day_1011/Exploring_Data_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Day 1

In [ ]:
import requests
import pandas as pd
import gc

In [ ]:
url = "https://jsonplaceholder.typicode.com/posts"
response = requests.get(url)
data = response.json()

In [ ]:
df = pd.DataFrame(data)
df = df[['userId', 'id', 'title']]
df = df.rename(columns={'id': 'postId'})

In [ ]:
df

,userId,postId,title
0,1,1,sunt aut facere repellat provident occaecati e...
1,1,2,qui est esse
2,1,3,ea molestias quasi exercitationem repellat qui...
3,1,4,eum et est occaecati
4,1,5,nesciunt quas odio
...,...,...,...
95,10,96,quaerat velit veniam amet cupiditate aut numqu...
96,10,97,quas fugiat ut perspiciatis vero provident
97,10,98,laboriosam dolor voluptates
98,10,99,temporibus sit alias delectus eligendi possimu...


index=False → tells pandas not to write the row indices (the numbers 0, 1, 2, …) into the CSV file

In [ ]:
df.to_csv('posts_clean.csv', index=False)

In [ ]:
df.head()

,userId,postId,title
0,1,1,sunt aut facere repellat provident occaecati e...
1,1,2,qui est esse
2,1,3,ea molestias quasi exercitationem repellat qui...
3,1,4,eum et est occaecati
4,1,5,nesciunt quas odio


In [1]:
import gc

def free_memory():
    gc.collect()

In [ ]:
free_memory()

## Day 2

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
data = {
    "customer_id": [1, 2, 3, 4, 5],
    "order_amount": [250, 300, 450, 270, 320],
    "region": ["East", "West", "North", "East", "South"],
    "churned": [0, 0, 1, 0, 1]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df_clean = df.dropna()
print("-------- Data Engineer - Cleaned Dataset --------")
print(df_clean.to_string(index = False))
print('\n')

-------- Data Engineer - Cleaned Dataset --------
 customer_id  order_amount region  churned
           1           250   East        0
           2           300   West        0
           3           450  North        1
           4           270   East        0
           5           320  South        1




In [ ]:
avg_order = df_clean['order_amount'].mean()
sales_by_region = df_clean.groupby('region')['order_amount'].sum().reset_index()

In [ ]:
print("-------- Data Analyst - Insights --------")
print(f"Average order amount {avg_order:.2f}")
print('\nSales by region:')
print(sales_by_region.to_string(index = False))
print('\n')

-------- Data Analyst - Insights --------
Average order amount 318.00

Sales by region:
region  order_amount
  East           520
 North           450
 South           320
  West           300




In [ ]:
X = df_clean[['order_amount']]
y = df_clean['churned']

In [ ]:
model = LogisticRegression()
model.fit(X, y)
prediction = model.predict([[500]])[0]

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
print("-------- Data Scientist - Prediction --------")
print(f"Prediction for a customer with an order amount of 500: {prediction} \
(0: Not churn, 1: Churn)")

-------- Data Scientist - Prediction --------
Prediction for a customer with an order amount of 500: 1 (0: Not churn, 1: Churn)


In [ ]:
free_memory()

## Day 3

In [ ]:
import sqlite3

In [ ]:
# Create in-memory database
conn = sqlite3.connect(':memory:')

In [ ]:
data = {
    "customer_id": [1, 2, 3, 4, 5],
    "order_amount": [250, 300, 450, 270, 320],
    "region": ["East", "West", "North", "East", "South"],
    "churned": [0, 0, 1, 0, 1]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
# Load data into SQLite (simulating a small warehouse)
df.to_sql('orders', conn, index=False, if_exists='replace')

5

In [ ]:
query = "SELECT region, AVG(order_amount) as avg_order FROM orders GROUP BY region"
result = pd.read_sql_query(query, conn)

In [ ]:
print(result)

  region  avg_order
0   East      260.0
1  North      450.0
2  South      320.0
3   West      300.0


## Day 4

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect(":memory:")
orders = pd.DataFrame({
    "customer_id": [1, 2, 3, 1, 5],
    "order_id": [101, 104, 110, 107, 108],
    "order_amount": [250, 300, 450, 270, 320],
    "region": ["East", "West", "North", "East", "South"],
    "churned": [0, 0, 1, 0, 1]
})
orders.to_sql("orders", conn, index=False, if_exists="replace")

5

In [ ]:
print("Database (OLTP) - Get customer 1 orders")
print(pd.read_sql_query("SELECT * FROM orders WHERE customer_id = 1", conn))

Database (OLTP) - Get customer 1 orders
   customer_id  order_id  order_amount region  churned
0            1       101           250   East        0
1            1       107           270   East        0


In [ ]:
# Simulating a warehouse OLAP
print('\nWarehouse (OLAP) - Aggregate sales by customer')
print(pd.read_sql_query("SELECT customer_id, SUM(order_amount) as total_sales FROM orders GROUP BY customer_id", conn))


Warehouse (OLAP) - Aggregate sales by customer
   customer_id  total_sales
0            1          520
1            2          300
2            3          450
3            5          320


In [ ]:
# Simulating Data Lake
print("Data Lake - Store raw JSON")
raw_data = [
    {"event": "page_view", "user": 2, "page": "/home"},
    {"event": "click", "user": 3, "page": "buy"}
]
print(raw_data)

Data Lake - Store raw JSON
[{'event': 'page_view', 'user': 2, 'page': '/home'}, {'event': 'click', 'user': 3, 'page': 'buy'}]


In [ ]:
free_memory()

## Day 5

In [ ]:
import pandas as pd
import time

In [ ]:
orders = [
    {"order_id": 1, "amount": 250},
    {"order_id": 2, "amount": 520},
    {"order_id": 3, "amount": 670},
    {"order_id": 4, "amount": 400},
    {"order_id": 5, "amount": 350},
    {"order_id": 6, "amount": 200}
]

In [ ]:
df = pd.DataFrame(orders)
print("Batch Processing - Total Orders")
print(df["amount"].sum())

Batch Processing - Total Orders
2390


In [ ]:
print("Stream Processing - Processing as data arrives")
total = 0
for order in orders:
    total += order["amount"]
    print(f"Processed order {order['order_id']} | Running total = {total}")
    time.sleep(1)

Stream Processing - Processing as data arrives
Processed order 1 | Running total = 250
Processed order 2 | Running total = 770
Processed order 3 | Running total = 1440
Processed order 4 | Running total = 1840
Processed order 5 | Running total = 2190
Processed order 6 | Running total = 2390


In [ ]:
free_memory()

## Day 6

In [ ]:
import pandas as pd

In [ ]:
raw_data = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5],
    "amount": ["270", "320", "440", "200", "310"]
})
print("Raw data: ")
print(raw_data)

Raw data: 
   order_id amount
0         1    270
1         2    320
2         3    440
3         4    200
4         5    310


In [ ]:
etl_transformed = raw_data.copy()
etl_transformed["amount"] = pd.to_numeric(etl_transformed["amount"].str.strip(), errors="coerce").fillna(0)
etl_transformed.to_csv("etl_clean.csv", index = False)
print("\nETL Transformation: Transformed before loading")
print(etl_transformed)


ETL Transformation: Transformed before loading
   order_id  amount
0         1     270
1         2     320
2         3     440
3         4     200
4         5     310


In [ ]:
raw_data.to_csv("elt_raw.csv", index = False)

In [ ]:
elt_data = pd.read_csv("elt_raw.csv", dtype={"amount": str})
elt_data["amount"] = pd.to_numeric(elt_data["amount"].str.strip(), errors="coerce").fillna(0)
print("ELT Data: Transformed after loading")
print(elt_data)

ELT Data: Transformed after loading
   order_id  amount
0         1     270
1         2     320
2         3     440
3         4     200
4         5     310


In [ ]:
free_memory()

## Day 7

In [ ]:
import time

In [ ]:
def extract():
  print("Extracting data")
  time.sleep(1)
  return [150, 220, 170]

In [ ]:
def transform_data(data):
  print("Transforming data")
  time.sleep(1)
  return [x * 2 for x in data]

In [ ]:
def load_data(data):
  print("Loading data")
  time.sleep(1)
  return ("Final data: ", data)

In [ ]:
data = extract()
transformed = transform_data(data)
load_data(transformed)

Extracting data
Transforming data
Loading data


('Final data: ', [300, 440, 340])

In [2]:
free_memory()

## Day 8

In [3]:
import pandas as pd

In [4]:
sales = pd.DataFrame({
    "sales_id": [1,2,3,4,5,6],
    "customer_id": [101, 103, 104, 110, 109, 102],
    "date_id": [1001, 1004, 1007, 1002, 1002, 1001],
    "product_id": [200, 205, 204, 203, 208, 202],
    "quantity": [1, 2, 1, 4, 2, 3]
})

customers = pd.DataFrame({
    "customer_id": [101, 102, 103, 104, 105, 110, 109],
    "name": ["Alicia", "Ana", "Ben", "Cathy", "Cindy", "Mat", "Zach"],
    "region": ["North", "East", "East", "North", "South", "West", "West"],
    "age-group": ["25-34", "35-44", "25-34", "25-34", "45-54", "35-44","25-34"]
})

product = pd.DataFrame({
    "product_id": [200, 202, 203, 204, 205, 208],
    "product": ["Laptop", "Earphone", "Charger", "Phone case", "Phone", "Tablet"],
    "Category": ["Electronics", "Electronics","Electronics","Electronics","Electronics","Electronics"],
    "price": [1200, 400, 100, 60, 1000, 950]
})

dates = pd.DataFrame({
    "date_id": [1001, 1002, 1004, 1007],
    "date": pd.to_datetime(["2023-01-01", "2023-01-02", "2023-01-04", "2023-01-07"]),
    "month": ["January", "January", "January", "January"],
    "weekdays": ["Monday", "Tuesday", "Thursday", "Sunday"]
})

In [5]:
star_schema = sales.merge(customers, on="customer_id")\
                   .merge(product, on="product_id")\
                   .merge(dates, on="date_id")

In [6]:
star_schema["revenue"] = star_schema["quantity"] * star_schema["price"]

In [7]:
star_schema.head()

,sales_id,customer_id,date_id,product_id,quantity,name,region,age-group,product,Category,price,date,month,weekdays,revenue
0,1,101,1001,200,1,Alicia,North,25-34,Laptop,Electronics,1200,2023-01-01,January,Monday,1200
1,2,103,1004,205,2,Ben,East,25-34,Phone,Electronics,1000,2023-01-04,January,Thursday,2000
2,3,104,1007,204,1,Cathy,North,25-34,Phone case,Electronics,60,2023-01-07,January,Sunday,60
3,4,110,1002,203,4,Mat,West,35-44,Charger,Electronics,100,2023-01-02,January,Tuesday,400
4,5,109,1002,208,2,Zach,West,25-34,Tablet,Electronics,950,2023-01-02,January,Tuesday,1900


## Day 9

In [1]:
import pandas as pd

In [2]:
# Fact: Sales
sales = pd.DataFrame({
    "sale_id": [1,2,3,4,5,6],
    "customer_id": [101, 103, 104, 102, 101, 102],
    "product_id": [200, 205, 204, 203, 208, 202],
    "date": pd.to_datetime(["2023-01-01", "2023-01-02", "2023-01-04", "2023-01-04", "2023-01-07", "2023-01-07"]),
    "quantity": [1, 2, 1, 4, 2, 3]
})

# Dimension
customers = pd.DataFrame({
    "customer_id": [101, 102, 103, 104, 105],
    "name": ["Alicia", "Ana", "Ben", "Cathy", "Cindy"],
    "region": ["North", "East", "East", "North", "South"],
})

products = pd.DataFrame({
    "product_id": [200, 202, 203, 204, 208],
    "product": ["Laptop", "Earphone", "Charger", "Phone case", "Phone"],
    "category": ["Electronics", "Electronics","Electronics","Electronics","Electronics"],
    "price": [1200, 400, 100, 75, 1000]
})


In [3]:
# Join fact + dimensions
warehouse_view = sales.merge(customers, on="customer_id")\
                       .merge(products, on="product_id")

# Example analytics : Revenue by region
warehouse_view["revenue"] = warehouse_view["quantity"] * warehouse_view["price"]
print(warehouse_view.groupby("region")["revenue"].sum())

region
East     1600
North    3275
Name: revenue, dtype: int64


## Day 11

In [4]:
import time
import pandas as pd
import random
from datetime import datetime

In [5]:
batch_clicks = pd.DataFrame({
    "used_id": [101, 102, 103, 104, 105],
    "page": ["home", "home", "about", "contact", "home"],
    "timestamp": pd.date_range(start="2023-01-01", periods=5, freq="H")
})

print("\n ---- Batch Ingestion (Daily Clicks) -----")
print(batch_clicks)


 ---- Batch Ingestion (Daily Clicks) -----
   used_id     page           timestamp
0      101     home 2023-01-01 00:00:00
1      102     home 2023-01-01 01:00:00
2      103    about 2023-01-01 02:00:00
3      104  contact 2023-01-01 03:00:00
4      105     home 2023-01-01 04:00:00


/tmp/ipython-input-3909802934.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "timestamp": pd.date_range(start="2023-01-01", periods=5, freq="H")


In [6]:
pages = ["home", "search", "car", "product", "checkout"]

print("\n ---- Streaming Ingestion (Live User Events) -----")
for i in range(5):
  event = {
      "user_id": random.randint(101, 105),
      "page": random.choice(pages),
      "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  }
  print(f"New Event: {event}")
  time.sleep(1)


 ---- Streaming Ingestion (Live User Events) -----
New Event: {'user_id': 103, 'page': 'car', 'timestamp': '2025-09-09 00:58:29'}
New Event: {'user_id': 105, 'page': 'product', 'timestamp': '2025-09-09 00:58:30'}
New Event: {'user_id': 104, 'page': 'home', 'timestamp': '2025-09-09 00:58:31'}
New Event: {'user_id': 104, 'page': 'search', 'timestamp': '2025-09-09 00:58:32'}
New Event: {'user_id': 101, 'page': 'search', 'timestamp': '2025-09-09 00:58:33'}
